# ИМПОРТИРОВАНИЕ БИБЛИОТЕК

In [164]:
from vosk import Model, KaldiRecognizer

import sys
import os
import time
import shutil

import wave
import soundfile
import json

import pandas as pd

import spacy
from spacy.util import minibatch, compounding
from spacy.training.example import Example
import random
from sklearn.metrics import confusion_matrix

In [31]:
model = Model(r'vosk-model-ru-0.42')

# ПЕРЕРАСПРЕДЕЛЕНИЕ ФАЙЛОВ НА 7 ПАПОК

In [25]:
test_csv = pd.read_csv(r'dataset\test.csv')

In [26]:
test_csv.head(5)

,name,path,emotion,text
0,27_neutral_fear_n_100,neutral_fear_27/27_neutral_fear_n_100.wav,neutral,"Вам дадут целый минимальный оклад, но при этом..."
1,08_sadness_anger a_010,08_sadness_anger/08_sadness_anger a_010.wav,anger,Сколько можно звонить?
2,26_enthusiasm_happiness_e_120,enthusiasm_happiness_26/26_enthusiasm_happines...,enthusiasm,А как долго тебе нужно это всё узнавать?
3,42_anger_fear_a_190,anger_fear_42/42_anger_fear_a_190.wav,anger,Ну а мне в 5 часов вставать на работу!
4,04_fear_enthusiasm f_090,04_fear_enthusiasm/04_fear_enthusiasm f_090.wav,fear,"Честно, я не подскажу, ну как и обычно, любым ..."


In [28]:
for dirname, dirnames, filenames in os.walk(r'dataset\test'):
    
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        
        if file_path[-3:] == 'wav':
            file_emotion = test_csv['emotion'][test_csv['name'] == filename[:-4]].values[0]
            
            try:
                if file_emotion != '':
                    shutil.move(file_path, os.path.join(r'dataset\test', file_emotion))
            except:
                continue

# ПЕРЕКОДИРОВКА АУДИО В 16 БИТ

In [30]:
for dirname, dirnames, filenames in os.walk(r'dataset\train'):
    
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        
        if file_path[-3:] == 'wav':
            data, samplerate = soundfile.read(file_path)
            soundfile.write(file_path, data, samplerate, subtype='PCM_16')

# АУДИО В ТЕКСТ

In [35]:
for dirname, dirnames, filenames in os.walk(r'dataset\train'):
    
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        
        if file_path[-3:] == 'wav':
            wf = wave.open(file_path, 'rb')
            rec = KaldiRecognizer(model, wf.getframerate())
            
            result = ''
            last_n = False

            while True:
                data = wf.readframes(1000)
                if len(data) == 0:
                    break

                if rec.AcceptWaveform(data):
                    res = json.loads(rec.Result())

                    if res['text'] != '':
                        result += f" {res['text']}"
                        last_n = False
                    elif not last_n:
                        result += '\n'
                        last_n = True

            res = json.loads(rec.FinalResult())
            result += f" {res['text']}"
            
            with open(file_path[:-3] + 'txt', "w") as txt_file:
                txt_file.write(result)

In [221]:
# ДЛЯ ОДНОГО ФАЙЛА

file_path = r'dataset\train\happiness\03_disgust_neutral d_070.wav'

wf = wave.open(file_path, 'rb')
rec = KaldiRecognizer(model, wf.getframerate())
            
result = ''
last_n = False

while True:
    data = wf.readframes(1000)
    if len(data) == 0:
        break

    if rec.AcceptWaveform(data):
        res = json.loads(rec.Result())

        if res['text'] != '':
            result += f" {res['text']}"
            last_n = False
        elif not last_n:
            result += '\n'
            last_n = True

res = json.loads(rec.FinalResult())
result += f" {res['text']}"

print(result)

 я пошла на новое обучение по инстаграмму и даже уже заплатила за него хотя оно стоило очень дорого но родители мне помогли и теперь я буду супер специалистом


In [36]:
import winsound
winsound.Beep(1000, 3000)

# ОЦЕНКА ТОЧНОСТИ РАСПОЗНАВАНИЯ РЕЧИ

In [37]:
train_csv = pd.read_csv(r'dataset\train.csv')

In [38]:
train_csv.head(5)

,name,path,emotion,text
0,27_neutral_fear_n_100,neutral_fear_27/27_neutral_fear_n_100.wav,neutral,"Вам дадут целый минимальный оклад, но при этом..."
1,08_sadness_anger a_010,08_sadness_anger/08_sadness_anger a_010.wav,anger,Сколько можно звонить?
2,26_enthusiasm_happiness_e_120,enthusiasm_happiness_26/26_enthusiasm_happines...,enthusiasm,А как долго тебе нужно это всё узнавать?
3,42_anger_fear_a_190,anger_fear_42/42_anger_fear_a_190.wav,anger,Ну а мне в 5 часов вставать на работу!
4,04_fear_enthusiasm f_090,04_fear_enthusiasm/04_fear_enthusiasm f_090.wav,fear,"Честно, я не подскажу, ну как и обычно, любым ..."


In [60]:
from difflib import SequenceMatcher

true_words = 0
all_words = 0

number = 0
summ = 0

for dirname, dirnames, filenames in os.walk(r'dataset\train'):
    
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        
        if file_path[-3:] == 'txt':
            with open(file_path, 'r') as txt_file:
                text = txt_file.read()
                number += 1

                summ += SequenceMatcher(a=text, b=train_csv['text'][train_csv['name'] == filename[:-4]].values[0]).ratio()
                
print(summ / number)

0.8603738484841921


# КЛАССИФИКАЦИЯ ТЕКСТА

In [130]:
main_labels = ['anger', 'disgust', 'enthusiasm', 'fear', 'happiness', 'neutral', 'sadness']

In [168]:
def load_training_data(data_directory=r'dataset\train', split=0.75, limit=0) -> tuple:

    reviews = []
    
    for label in main_labels:
        labeled_directory = os.path.join(data_directory, label)
        
        for review in os.listdir(labeled_directory):
            if review.endswith('.txt'):
                
                with open(os.path.join(labeled_directory, review)) as f:
                    text = f.read()
                    if text.strip():
                        spacy_label = {
                            'cats': {
                                'anger' : 'anger' == label,
                                'disgust' : 'disgust' == label,
                                'enthusiasm' : 'enthusiasm' == label,
                                'fear' : 'fear' == label,
                                'happiness' : 'happiness' == label,
                                'neutral' : 'neutral' == label,
                                'sadness' : 'sadness' == label
                            }
                        }
                        reviews.append((text, spacy_label))
                        
        random.shuffle(reviews)
        random.shuffle(reviews)
        random.shuffle(reviews)

    if limit:
        reviews = reviews[:limit]
    split = int(len(reviews) * split)
        
    return reviews[:split], reviews[split:]

In [173]:
def evaluate_model(tokenizer, textcat, valid_data):
    reviews, labels = zip(*valid_data)
    reviews = (tokenizer(review) for review in reviews)
    
    first_true, second_pred = [], []
    
    for i, review in enumerate(textcat.pipe(reviews)):
        true_label = labels[i]['cats']
        score_pos = review.cats
        
        for i in main_labels:
            if true_label[i]:
                first_true.append(i)
        
        maxx = 0
        for i in main_labels:
            if score_pos[i] > maxx:
                temp = i
                maxx = score_pos[i]
        second_pred.append(temp)
                

    matrix = confusion_matrix(first_true, second_pred)
    
    # print(matrix)

    precision, recall, F1_score = [], [], []
    for i in range(len(matrix)):
        precision.append(matrix[i, i] / sum(matrix[:, i]))
        recall.append(matrix[i, i] / sum(matrix[i, :]))
        if precision[i] + recall[i] == 0:
            F1_score.append(0)
        else:
            F1_score.append(2 * precision[i] * recall[i] / (precision[i] + recall[i]))
    
    return {'precision': sum(precision) / 7, 'recall': sum(recall) / 7, 'F1-score': sum(F1_score) / 7}

In [177]:
def train_model(training_data, valid_data, iterations=20):
    
    nlp = spacy.load('ru_core_news_sm')
    
    if 'textcat' not in nlp.pipe_names:
        textcat = nlp.add_pipe('textcat', last=True)
    else:
        textcat = nlp.get_pipe('textcat')
    
    for i in main_labels:
        textcat.add_label(i)

    training_excluded_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
    
    with nlp.disable_pipes(training_excluded_pipes):
        optimizer = nlp.begin_training()

        print('Начинаем обучение')
        print('Loss\t\tPrec.\tRec.\tF1-score')
        batch_sizes = compounding(32.0, 64.0, 2)
        
        for i in range(iterations):
            loss = {}
            random.shuffle(training_data)
            batches = minibatch(training_data, size=batch_sizes)
            
            for batch in batches:
                for text, labels in batch:

                    doc = nlp.make_doc(text)
                    example = Example.from_dict(doc, labels)

                    nlp.update([example], losses=loss, drop=0.5, sgd=optimizer)
                    
                
            with textcat.model.use_params(optimizer.averages):
                evaluation_results = evaluate_model(
                    tokenizer=nlp.tokenizer,
                    textcat=textcat,
                    valid_data=valid_data
                )
                
                print(f"{loss['textcat']:9.6f}\t\
{evaluation_results['precision']:.3f}\t\
{evaluation_results['recall']:.3f}\t\
{evaluation_results['F1-score']:.3f}")
                

    with nlp.use_params(optimizer.averages):
        nlp.to_disk('model_artifacts')

In [178]:
train_data, valid_data = load_training_data(data_directory=r'dataset\train', split=0.75, limit=0)
test_data = load_training_data(data_directory=r'dataset\test', split=1.0, limit=0)[0]

In [179]:
train_model(train_data, valid_data, iterations=60)

Начинаем обучение
Loss		Prec.	Rec.	F1-score
[[33  5  2  2  3  3  0]
 [18  4  1  1  4  7  1]
 [22  5  1  2  4  9  3]
 [21  5  0  2  3 10  0]
 [22  7  0  1  7  3  0]
 [15  4  0  0  5  9  2]
 [16  4  1  0  3  6  3]]
103.999564	0.223	0.199	0.163
[[10 10  4 14 10  0  0]
 [ 7 13  5  4  6  0  1]
 [11  8  8  7 12  0  0]
 [ 6 17  2  6 10  0  0]
 [ 3  7 11  2 16  0  1]
 [ 8  9  7  2  8  0  1]
 [ 5 12  4  6  2  0  4]]
103.433503	nan	0.202	nan


C:\Users\MGeorge\AppData\Local\Temp\ipykernel_4292\781364747.py:29: RuntimeWarning: invalid value encountered in scalar divide
  precision.append(matrix[i, i] / sum(matrix[:, i]))


[[ 8  3  4 20  6  7  0]
 [ 6  3  0 15  6  6  0]
 [ 6  2  1 17  9  9  2]
 [ 1  2  1 27  5  5  0]
 [ 2  0  5 11 10 12  0]
 [ 0  2  5  9  6 13  0]
 [ 4  0  0 18  8  3  0]]
100.634767	0.182	0.222	0.175
[[17  8  1 10  4  8  0]
 [15  1  0  5  5  8  2]
 [13  3  3  8  6 11  2]
 [ 7  7  1 17  4  2  3]
 [ 6  6  1  1  9 17  0]
 [ 9  2  2  4  4 12  2]
 [ 7  3  0 10  5  5  3]]
96.377152	0.233	0.217	0.198
[[19 10  5  5  7  1  1]
 [ 9  8  3  2 12  1  1]
 [10  8 10  3 10  5  0]
 [ 9 10  1 13  6  0  2]
 [ 4  6  1  3 20  6  0]
 [11  2  1  0 10 10  1]
 [ 7  9  0  2  8  2  5]]
91.006512	0.363	0.299	0.300
[[14 12  6  7  5  3  1]
 [ 8  9  5  3  4  5  2]
 [ 8 10 12  2 11  2  1]
 [ 7 10  2 11  4  4  3]
 [ 3  7  7  2 13  7  1]
 [ 6  4  4  1  7 11  2]
 [ 4  9  0  4  6  5  5]]
86.486226	0.288	0.266	0.268
[[16 13  7  6  4  2  0]
 [ 7 10  8  2  3  3  3]
 [ 5 12 11  4  7  5  2]
 [ 9 11  1 10  2  3  5]
 [ 3  3  8  2 12  9  3]
 [ 4  4  3  7  2 11  4]
 [ 5  9  0  5  7  3  4]]
76.770966	0.268	0.261	0.261
[[13 13  6  6 

[[16 10  9  4  2  6  1]
 [ 6  7  9  5  3  3  3]
 [10  4  8  4  7  8  5]
 [ 8  6  5 10  3  6  3]
 [ 5  2  7  2  8 11  5]
 [ 0  8  8  2  2 11  4]
 [ 3  4  4  7  4  4  7]]
 2.591578	0.244	0.239	0.239
[[15 10  9  5  2  6  1]
 [ 6  8  8  5  4  1  4]
 [ 8  4  8  4  8  9  5]
 [ 7  6  6 10  3  6  3]
 [ 4  2  5  2  8 12  7]
 [ 1  6  8  2  1 13  4]
 [ 3  4  4  7  4  4  7]]
 1.746948	0.249	0.248	0.246
[[15 10  9  5  2  6  1]
 [ 6  7  8  5  5  1  4]
 [ 8  4  8  4  8  8  6]
 [ 7  7  7  9  3  6  2]
 [ 4  2  6  2  8 12  6]
 [ 0  6  9  3  1 13  3]
 [ 3  4  4  7  4  4  7]]
 1.421527	0.242	0.241	0.239
[[15 10  9  5  2  6  1]
 [ 6  7  8  5  5  1  4]
 [ 7  5  8  4  8  8  6]
 [ 7  7  7  9  3  6  2]
 [ 4  2  8  1  8 12  5]
 [ 0  6  9  2  2 12  4]
 [ 4  4  4  7  4  3  7]]
 2.784176	0.241	0.236	0.236
[[17  9 10  5  1  5  1]
 [ 7  7  8  4  5  1  4]
 [ 6  6  8  4  9  7  6]
 [ 9  6  6 10  2  6  2]
 [ 4  2  9  1  8 11  5]
 [ 1  5 10  3  2 11  3]
 [ 5  3  4  7  5  2  7]]
 1.932917	0.247	0.242	0.243
[[16  9 10  5  

In [216]:
def test_model_once(file_path):
    
    input_data = open(file_path).read()
    true_class = file_path[file_path.rfind('\\', 0, file_path.rfind('\\') - 1) + 1 : file_path.rfind('\\')]
    
    loaded_model = spacy.load("model_artifacts")
    
    parsed_text = loaded_model(input_data)
    
    maxx = 0
    for i in main_labels:
        if parsed_text.cats[i] > maxx:
            temp = i
            maxx = parsed_text.cats[i]
            
    print(f"Текст обзора: {input_data}\n\
Правильный класс: {true_class}\n\
Предсказание: {temp}\n\
Score: {maxx:.3f}\n")

In [217]:
test_model_once(r'dataset\test\anger\02_anger_sadness a_032.txt')
test_model_once(r'dataset\test\disgust\19_anger_disgust_d_100.txt')

test_model_once(r'dataset\test\sadness\12_sadness_disgust_s_070.txt')

Текст обзора:  чем ты занимался что тебя отвлекло от работы
Правильный класс: anger
Предсказание: anger
Score: 0.916

Текст обзора:  извините мы не можем вам помочь в этой ситуации
Правильный класс: disgust
Предсказание: disgust
Score: 0.989

Текст обзора:  почему не потянет я думаю если вы постараетесь то тогда он начнёт понимать лучший предмет и тогда вы можете поставить ему заслуженную тройку
Правильный класс: sadness
Предсказание: enthusiasm
Score: 0.960



In [218]:
def test_model(path):
    loaded_model = spacy.load("model_artifacts")
    
    y_true, y_pred = [], []
    
    for label in main_labels:
        labeled_directory = os.path.join(path, label)
        
        for review in os.listdir(labeled_directory):
            if review.endswith('.txt'):
                
                with open(os.path.join(labeled_directory, review)) as f:
                    text = f.read()
                    
                    parsed_text = loaded_model(text)
    
                    maxx = 0
                    for i in main_labels:
                        if parsed_text.cats[i] > maxx:
                            temp = i
                            maxx = parsed_text.cats[i]

                y_true.append(label)
                y_pred.append(temp)
    return y_true, y_pred
                
y_true, y_pred = test_model(r'dataset\test')
    
matrix = confusion_matrix(y_true, y_pred)

precision, recall, F1_score = [], [], []

for i in range(len(matrix)):
    precision.append(matrix[i, i] / sum(matrix[:, i]))
    recall.append(matrix[i, i] / sum(matrix[i, :]))
    F1_score.append(2 * precision[i] * recall[i] / (precision[i] + recall[i]))
    
print(' ' * 10, 'anger', 'disgust', 'enthusiasm', 'fear', 'happiness', 'neutral', 'sadness', sep='   ')

print('Precision', end='  ')
for i in range(len(precision)):
    print(f'{str(round(precision[i], 3)) : >7}', end='   ')
print()

print('Recall', end='     ')
for i in range(len(recall)):
    print(f'{str(round(recall[i], 3)) : >7}', end='   ')
print()
    
print('F1-score', end='   ')
for i in range(len(F1_score)):
    print(f'{str(round(F1_score[i], 3)) : >7}', end='   ')
print()

             anger   disgust   enthusiasm   fear   happiness   neutral   sadness
Precision    0.818     0.803      0.73     0.871     0.849      0.77     0.827   
Recall       0.818     0.819     0.753     0.831      0.81      0.83     0.802   
F1-score     0.818     0.811     0.741     0.851     0.829     0.799     0.814   


In [219]:
matrix

array([[144,   9,   8,   5,   1,   6,   3],
       [  7, 122,   6,   4,   5,   1,   4],
       [  7,   5, 119,   4,  10,   8,   5],
       [  7,   4,   6, 148,   2,   9,   2],
       [  5,   3,   8,   2, 141,  11,   4],
       [  2,   6,   9,   2,   3, 127,   4],
       [  4,   3,   7,   5,   4,   3, 105]], dtype=int64)